In [1]:
import json
from pathlib import Path
from collections import defaultdict

import requests
import pandas as pd

DATA_DIR = Path('../association/onek1k_website_data/')
OUTPUT_DIR = Path('../')

# API Clients

In [2]:
class ApiClient:
    """
    Base API client providing common cache operations.
    """
    def __init__(self, base_url, connection=None, cache=None):
        self.base_url = base_url
        self.connection = connection or requests
        self.cache = {}

        if isinstance(cache, (Path, str)):
            self.load_cache(file_path=cache)
        
    def save_cache(self, file_path):
        with open(file_path, 'wt') as file:
            json.dump(self.cache, file)
        return self
            
    def load_cache(self, file_path):
        with open(file_path, 'rt') as file:
            self.cache = json.load(file)
        return self

## MyVariant
Simple client for [MyVariant](https://myvariant.info/v1/api) service to retrieve dbSNP record information such as consequence etc.

In [3]:
class MyVariantClient(ApiClient):
    """
    Simple client for https://myvariant.info/v1/api
    """
    
    def __init__(self, base_url='http://myvariant.info/v1/api', connection=None, cache=None):
        super.__init__(self, base_url=base_url, connection=connection, cache=cache)
        
    def fetch_rsid(self, identifier):
        if identifier in self.cache:
            return self.cache[identifier]
        
        result = self.connection.get(f"{self.base_url}/variant/{identifier}").json()
        self.cache[identifier] = result
        return result

## HGNC
Simple client for [HGNC rest API](https://www.genenames.org/help/rest/) to retrieve gene information such as names, aliases, transcripts etc.

In [4]:
class HgncClient(ApiClient):   
    """
    Simple client for https://www.genenames.org/help/rest/
    """
        
    def __init__(self, base_url='http://rest.genenames.org', connection=None, cache=None):
        super().__init__(base_url=base_url, connection=connection, cache=cache)

    def fetch(self, identifier, fields=('hgnc_id')):
        if isinstance(fields, str):
            fields = (fields,)
            
        if identifier in self.cache.keys():
            field_mapping = self.cache[identifier]
            requested_fields = set(fields) & set(field_mapping.keys())
            for field in requested_fields:
                if field_mapping.get(field, None):
                    return field_mapping[field]
            if requested_fields == set(field_mapping.keys()):
                return None
        else:
            self.cache[identifier] = {}
        
        for field in fields:
            response = self.connection.get(
                f"{self.base_url}/fetch/{field}/{identifier}",
                headers={'Accept': 'application/json'}
            ).json()

            records = response['response'].get('docs', [])
            if not records:
                records = None
                
            self.cache[identifier][field] = records
            
            if records:
                break
        
        return records
    
    def fetch_many(self, identifiers, fields=('hgnc_id')):
        records = {}
        for identifier in identifiers:
            records[identifier] = None
            result = self.fetch(identifier, field=field)
            for field in fields:
                
                if result is not None:
                    records[identifier] = result
                    break
        return records

# Generate Gene search file
Generate `gene_search_terms.json.txt` file where each line consists of a valid `JSON` list element. Each of these elements is a tuple that looks like `[gene_name, [search_terms]]`.

In [5]:
gene_search_terms = defaultdict(list)
gene_metadata = {}

# ---- Manually insert info not available online
gene_metadata['AC007308.6'] = [{
    'ensembl_gene_id': 'ENSG00000234252',
    'symbol': 'AC007308.6',
    'name': 'AC007308.6 (Clone-based (Vega))',
}]
gene_metadata['AC002472.13'] = [{
    'ensembl_gene_id': 'ENSG00000187905',
    'symbol': 'AC002472.13',
    'name': 'Leucine-rich repeat-containing protein LOC400891',
}]
gene_metadata['CTA-29F11.1'] = [{
    'ensembl_gene_id': 'CTA-29F11.1',
    'symbol': 'CTA-29F11.1',
    'name': 'lncRNA',
}]
gene_metadata['AC000068.5'] = [{
    'ensembl_gene_id': 'ENSG00000185065',
    'symbol': 'AC000068.5',
    'name': 'AC000068.5 (Clone-based (Vega) gene)',
}]
# ----------------------------------------------

cache_path = OUTPUT_DIR / 'hgnc_cache.json'
if cache_path.exists():
    hgnc_client = HgncClient(cache=cache_path)
else:
    hgnc_client = HgncClient(cache=None)

cell_labels = sorted([
    expression_file.name.split('_')[0].strip() 
    for expression_file in DATA_DIR.glob('*_eQTLs.tsv')
])
    
for label in cell_labels:
    df = pd.read_csv(DATA_DIR / f"{label}_eQTLs.tsv", header=0, delimiter='\t')
    for gene, group in df.groupby('GENE'):
        print(f"Fetching metadata for '{gene}'")
        metadata = hgnc_client.fetch(
            identifier=gene,
            fields=('symbol', 'prev_symbol', 'alias_symbol', 'vega_id', 'uniprot_ids')
        )
        
        if metadata:
            gene_metadata[gene] = metadata
            gene_search_terms[gene].append(metadata[0]['ensembl_gene_id'])
            gene_search_terms[gene].append(metadata[0]['hgnc_id'])
            gene_search_terms[gene].append(metadata[0]['symbol'])
            gene_search_terms[gene].append(metadata[0]['name'])
        
        gene_search_terms[gene].append(label)
        gene_search_terms[gene].append(gene)
        gene_search_terms[gene] += list(group['SNP'])
        
hgnc_client.save_cache(OUTPUT_DIR / 'hgnc_cache.json')

with open(OUTPUT_DIR / 'gene_search_terms.json.txt', 'wt') as file:
    for gene, search_terms in gene_search_terms.items():
        file.write(json.dumps([gene, list(set(search_terms))]))
        file.write('\n')
        
with open(OUTPUT_DIR / 'gene_metadata.json', 'wt') as file:
    json.dump(gene_metadata, file)

Fetching metadata for 'ARSA'
Fetching metadata for 'C22orf34'
Fetching metadata for 'CHCHD10'
Fetching metadata for 'CRYBB2'
Fetching metadata for 'CTA-29F11.1'
Fetching metadata for 'APOBEC3C'
Fetching metadata for 'ARFGAP3'
Fetching metadata for 'BCR'
Fetching metadata for 'CHCHD10'
Fetching metadata for 'CRYBB2'
Fetching metadata for 'AC000068.5'
Fetching metadata for 'APOBEC3G'
Fetching metadata for 'APOL2'
Fetching metadata for 'ARSA'
Fetching metadata for 'BCR'
Fetching metadata for 'ADRBK2'
Fetching metadata for 'APOBEC3G'
Fetching metadata for 'APOBEC3H'
Fetching metadata for 'APOL6'
Fetching metadata for 'ARSA'
Fetching metadata for 'AC002472.13'
Fetching metadata for 'APOBEC3G'
Fetching metadata for 'APOBEC3H'
Fetching metadata for 'ARSA'
Fetching metadata for 'ASPHD2'
Fetching metadata for 'ARSA'
Fetching metadata for 'CTA-29F11.1'
Fetching metadata for 'DDT'
Fetching metadata for 'FAM118A'
Fetching metadata for 'GGT1'
Fetching metadata for 'DDT'
Fetching metadata for 'FAM11

# Generate gene feature JSON file
An array with a record for each gene containinig exon information and variant associations

In [6]:
import hail
from hail.experimental import import_gtf

# Big file... might take a few years
ht = import_gtf("../gencode.v19.annotation.gtf.gz", force=True)

Initializing Hail with default parameters...
Running on Apache Spark version 2.4.1
SparkUI available at http://172.20.10.5:4040
Welcome to
     __  __     <>__
    / /_/ /__  __/ /
   / __  / _ `/ / /
  /_/ /_/\_,_/_/_/   version 0.2.61-3c86d3ba497a
LOGGING: writing to /Users/daniel/git/cpg/exome-results-browsers/src/data/tob/notebooks/hail-20210815-1555-0.2.61-3c86d3ba497a.log
2021-08-15 15:55:15 Hail: INFO: Reading table without type imputation
  Loading field 'f0' as type str (not specified)
  Loading field 'f1' as type str (not specified)
  Loading field 'f2' as type str (not specified)
  Loading field 'f3' as type int32 (user-supplied)
  Loading field 'f4' as type int32 (user-supplied)
  Loading field 'f5' as type float64 (user-supplied)
  Loading field 'f6' as type str (not specified)
  Loading field 'f7' as type int32 (user-supplied)
  Loading field 'f8' as type str (not specified)
2021-08-15 15:56:21 Hail: INFO: wrote table with 2619444 rows in 1 partition to /tmp/ZoWsX7JqKuh9c

## Process hail table 
Extract exons, UTRs and CDS regions for each gene

## Process eQTL association files
Process the raw association files into a `JSON` mapping indexed by cell type and then further by gene name. Values are a list of SNP mappings containing p-values, log10 p-values, dbSNP identifiers and other analysis data.

In [22]:
ht.filter((ht.gene_name == 'APOBEC3G') & (ht.transcript_id == 'ENST00000407997.3')).show(100)

,,,,,,,,,,,,,,,,,,,,,,
source,feature,score,strand,frame,havana_gene,tag,transcript_type,gene_name,gene_id,transcript_status,gene_status,transcript_name,level,exon_number,gene_type,havana_transcript,transcript_id,protein_id,exon_id,ccdsid,ont,interval
str,str,float64,str,int32,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,"interval<struct{seqname: str, position: int32}>"
"""HAVANA""","""UTR""",NA,"""+""",NA,"""OTTHUMG00000151081.4""","""basic""","""protein_coding""","""APOBEC3G""","""ENSG00000239713.3""","""KNOWN""","""KNOWN""","""APOBEC3G-001""","""2""",NA,"""protein_coding""","""OTTHUMT00000321219.1;""","""ENST00000407997.3""","""ENSP00000385057.3""",NA,"""CCDS13984.1""",NA,"[{seqname: chr22, position: 39473010}-{seqname: chr22, position: 39473366}]"
"""HAVANA""","""exon""",NA,"""+""",NA,"""OTTHUMG00000151081.4""","""basic""","""protein_coding""","""APOBEC3G""","""ENSG00000239713.3""","""KNOWN""","""KNOWN""","""APOBEC3G-001""","""2""","""1""","""protein_coding""","""OTTHUMT00000321219.1;""","""ENST00000407997.3""","""ENSP00000385057.3""","""ENSE00001809576.1""","""CCDS13984.1""",NA,"[{seqname: chr22, position: 39473010}-{seqname: chr22, position: 39473383}]"
"""HAVANA""","""transcript""",NA,"""+""",NA,"""OTTHUMG00000151081.4""","""basic""","""protein_coding""","""APOBEC3G""","""ENSG00000239713.3""","""KNOWN""","""KNOWN""","""APOBEC3G-001""","""2""",NA,"""protein_coding""","""OTTHUMT00000321219.1;""","""ENST00000407997.3""","""ENSP00000385057.3""",NA,"""CCDS13984.1""",NA,"[{seqname: chr22, position: 39473010}-{seqname: chr22, position: 39483748}]"
"""HAVANA""","""start_codon""",NA,"""+""",0,"""OTTHUMG00000151081.4""","""basic""","""protein_coding""","""APOBEC3G""","""ENSG00000239713.3""","""KNOWN""","""KNOWN""","""APOBEC3G-001""","""2""","""1""","""protein_coding""","""OTTHUMT00000321219.1;""","""ENST00000407997.3""","""ENSP00000385057.3""","""ENSE00001809576.1""","""CCDS13984.1""",NA,"[{seqname: chr22, position: 39473367}-{seqname: chr22, position: 39473369}]"
"""HAVANA""","""CDS""",NA,"""+""",0,"""OTTHUMG00000151081.4""","""basic""","""protein_coding""","""APOBEC3G""","""ENSG00000239713.3""","""KNOWN""","""KNOWN""","""APOBEC3G-001""","""2""","""1""","""protein_coding""","""OTTHUMT00000321219.1;""","""ENST00000407997.3""","""ENSP00000385057.3""","""ENSE00001809576.1""","""CCDS13984.1""",NA,"[{seqname: chr22, position: 39473367}-{seqname: chr22, position: 39473383}]"
"""HAVANA""","""exon""",NA,"""+""",NA,"""OTTHUMG00000151081.4""","""basic""","""protein_coding""","""APOBEC3G""","""ENSG00000239713.3""","""KNOWN""","""KNOWN""","""APOBEC3G-001""","""2""","""2""","""protein_coding""","""OTTHUMT00000321219.1;""","""ENST00000407997.3""","""ENSP00000385057.3""","""ENSE00003566101.1""","""CCDS13984.1""",NA,"[{seqname: chr22, position: 39474937}-{seqname: chr22, position: 39475090}]"
"""HAVANA""","""CDS""",NA,"""+""",1,"""OTTHUMG00000151081.4""","""basic""","""protein_coding""","""APOBEC3G""","""ENSG00000239713.3""","""KNOWN""","""KNOWN""","""APOBEC3G-001""","""2""","""2""","""protein_coding""","""OTTHUMT00000321219.1;""","""ENST00000407997.3""","""ENSP00000385057.3""","""ENSE00003566101.1""","""CCDS13984.1""",NA,"[{seqname: chr22, position: 39474937}-{seqname: chr22, position: 39475090}]"
"""HAVANA""","""exon""",NA,"""+""",NA,"""OTTHUMG00000151081.4""","""basic""","""protein_coding""","""APOBEC3G""","""ENSG00000239713.3""","""KNOWN""","""KNOWN""","""APOBEC3G-001""","""2""","""3""","""protein_coding""","""OTTHUMT00000321219.1;""","""ENST00000407997.3""","""ENSP00000385057.3""","""ENSE00003462837.1""","""CCDS13984.1""",NA,"[{seqname: chr22, position: 39476938}-{seqname: chr22, position: 39477232}]"


In [37]:
genes = []
association_file_suffix = 'eQTLs'

for gene in gene_metadata.keys():
    print(f"Gathering region data for {gene}")
    
    data = {
        'symbol': gene,
        'gene_id': None,
        'start': None,
        'stop': None,
        'chrom': None,
        'exons': [],
    }
    
    gene_rows = ht.filter(
        (ht.gene_name == gene) &
        (ht.transcript_name == f"{gene}-001") &
        (ht.source == 'ENSEMBL')
    )
    
    if gene_rows.count() == 0:
        gene_rows = ht.filter(
            (ht.gene_name == gene) &
            (ht.transcript_name == f"{gene}-001")
        )
    
    print(f"\tFound {gene_rows.count()} hits")
    print(gene_rows.show())
    
    for row in gene_rows.collect():
        if row.feature == 'transcript':
            data['gene_id'] = row.gene_id
            data['start'] = row.interval.start.position
            data['stop'] = row.interval.end.position
            data['chrom'] = row.interval.start.seqname.replace('chr', '')
        
        if row.feature in ('CDS', 'exon', 'UTR', 'start_codon', 'stop_codon'):
            data['exons'].append(
                {
                    'feature_type': row.feature,
                    'start': row.interval.start.position,
                    'stop': row.interval.end.position
                }
            )
            
    data['start'] = min(data['start'], sorted(data['exons'], key=lambda e: e['start'])[0]['start'])
    data['stop'] =  max(data['stop'], sorted(data['exons'], key=lambda e: e['stop'])[-1]['stop'])
    data['exons'] = [e for e in data['exons'] if e['feature_type'] in ('CDS', 'exon', 'UTR')]
            
    associations = []
    for label in cell_labels:
        df = pd.read_table(
            DATA_DIR / f"{label}_{association_file_suffix}.tsv", 
            header=0, 
            delimiter='\t'
        )

        records = df[df['GENE'] == gene].to_dict(orient='records')
        for r in records:
            r['chr'] = str(r['CHR'])
            r['id'] = f"{r['CHR']}-{r['BP']}-{r['A1']}-{r['A2']}"
            r['cell'] = label

        records = [{k.lower(): v for (k, v) in r.items()} for r in records]
        associations += records
    
    print(f"\tFound {len(associations)} variant associations")
    data['associations'] = associations
    genes.append(data)
    
json.dump(genes, open(OUTPUT_DIR / 'genes.json', 'wt'))

Gathering region data for AC007308.6
	Found 3 hits


,,,,,,,,,,,,,,,,,,,,,,
source,feature,score,strand,frame,havana_gene,tag,transcript_type,gene_name,gene_id,transcript_status,gene_status,transcript_name,level,exon_number,gene_type,havana_transcript,transcript_id,protein_id,exon_id,ccdsid,ont,interval
str,str,float64,str,int32,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,"interval<struct{seqname: str, position: int32}>"
"""HAVANA""","""exon""",NA,"""+""",NA,"""OTTHUMG00000153507.1""","""basic""","""antisense""","""AC007308.6""","""ENSG00000234252.1""","""KNOWN""","""NOVEL""","""AC007308.6-001""","""2""","""1""","""antisense""","""OTTHUMT00000331441.1;""","""ENST00000430719.1""",NA,"""ENSE00001767229.1""",NA,NA,"[{seqname: chr22, position: 21077335}-{seqname: chr22, position: 21078054}]"
"""HAVANA""","""transcript""",NA,"""+""",NA,"""OTTHUMG00000153507.1""","""basic""","""antisense""","""AC007308.6""","""ENSG00000234252.1""","""KNOWN""","""NOVEL""","""AC007308.6-001""","""2""",NA,"""antisense""","""OTTHUMT00000331441.1;""","""ENST00000430719.1""",NA,NA,NA,NA,"[{seqname: chr22, position: 21077335}-{seqname: chr22, position: 21080208}]"
"""HAVANA""","""exon""",NA,"""+""",NA,"""OTTHUMG00000153507.1""","""basic""","""antisense""","""AC007308.6""","""ENSG00000234252.1""","""KNOWN""","""NOVEL""","""AC007308.6-001""","""2""","""2""","""antisense""","""OTTHUMT00000331441.1;""","""ENST00000430719.1""",NA,"""ENSE00001674352.1""",NA,NA,"[{seqname: chr22, position: 21078529}-{seqname: chr22, position: 21080208}]"


None
	Found 4 variant associations
Gathering region data for AC002472.13
	Found 10 hits


,,,,,,,,,,,,,,,,,,,,,,
source,feature,score,strand,frame,havana_gene,tag,transcript_type,gene_name,gene_id,transcript_status,gene_status,transcript_name,level,exon_number,gene_type,havana_transcript,transcript_id,protein_id,exon_id,ccdsid,ont,interval
str,str,float64,str,int32,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,"interval<struct{seqname: str, position: int32}>"
"""HAVANA""","""exon""",NA,"""+""",NA,"""OTTHUMG00000150897.2""","""basic""","""processed_transcript""","""AC002472.13""","""ENSG00000187905.6""","""KNOWN""","""NOVEL""","""AC002472.13-001""","""2""","""1""","""protein_coding""","""OTTHUMT00000320470.1;""","""ENST00000497328.1""",NA,"""ENSE00003618695.1""",NA,NA,"[{seqname: chr22, position: 21400249}-{seqname: chr22, position: 21400414}]"
"""HAVANA""","""transcript""",NA,"""+""",NA,"""OTTHUMG00000150897.2""","""basic""","""processed_transcript""","""AC002472.13""","""ENSG00000187905.6""","""KNOWN""","""NOVEL""","""AC002472.13-001""","""2""",NA,"""protein_coding""","""OTTHUMT00000320470.1;""","""ENST00000497328.1""",NA,NA,NA,NA,"[{seqname: chr22, position: 21400249}-{seqname: chr22, position: 21418457}]"
"""HAVANA""","""exon""",NA,"""+""",NA,"""OTTHUMG00000150897.2""","""basic""","""processed_transcript""","""AC002472.13""","""ENSG00000187905.6""","""KNOWN""","""NOVEL""","""AC002472.13-001""","""2""","""2""","""protein_coding""","""OTTHUMT00000320470.1;""","""ENST00000497328.1""",NA,"""ENSE00003645500.1""",NA,NA,"[{seqname: chr22, position: 21401645}-{seqname: chr22, position: 21401787}]"
"""HAVANA""","""exon""",NA,"""+""",NA,"""OTTHUMG00000150897.2""","""basic""","""processed_transcript""","""AC002472.13""","""ENSG00000187905.6""","""KNOWN""","""NOVEL""","""AC002472.13-001""","""2""","""3""","""protein_coding""","""OTTHUMT00000320470.1;""","""ENST00000497328.1""",NA,"""ENSE00003473894.1""",NA,NA,"[{seqname: chr22, position: 21402173}-{seqname: chr22, position: 21403446}]"
"""HAVANA""","""exon""",NA,"""+""",NA,"""OTTHUMG00000150897.2""","""basic""","""processed_transcript""","""AC002472.13""","""ENSG00000187905.6""","""KNOWN""","""NOVEL""","""AC002472.13-001""","""2""","""4""","""protein_coding""","""OTTHUMT00000320470.1;""","""ENST00000497328.1""",NA,"""ENSE00003591684.1""",NA,NA,"[{seqname: chr22, position: 21406538}-{seqname: chr22, position: 21406647}]"
"""HAVANA""","""exon""",NA,"""+""",NA,"""OTTHUMG00000150897.2""","""basic""","""processed_transcript""","""AC002472.13""","""ENSG00000187905.6""","""KNOWN""","""NOVEL""","""AC002472.13-001""","""2""","""5""","""protein_coding""","""OTTHUMT00000320470.1;""","""ENST00000497328.1""",NA,"""ENSE00003570620.1""",NA,NA,"[{seqname: chr22, position: 21407649}-{seqname: chr22, position: 21407764}]"
"""HAVANA""","""exon""",NA,"""+""",NA,"""OTTHUMG00000150897.2""","""basic""","""processed_transcript""","""AC002472.13""","""ENSG00000187905.6""","""KNOWN""","""NOVEL""","""AC002472.13-001""","""2""","""6""","""protein_coding""","""OTTHUMT00000320470.1;""","""ENST00000497328.1""",NA,"""ENSE00003567574.1""",NA,NA,"[{seqname: chr22, position: 21409387}-{seqname: chr22, position: 21409465}]"
"""HAVANA""","""exon""",NA,"""+""",NA,"""OTTHUMG00000150897.2""","""basic""","""processed_transcript""","""AC002472.13""","""ENSG00000187905.6""","""KNOWN""","""NOVEL""","""AC002472.13-001""","""2""","""7""","""protein_coding""","""OTTHUMT00000320470.1;""","""ENST00000497328.1""",NA,"""ENSE00001941509.1""",NA,NA,"[{seqname: chr22, position: 21411225}-{seqname: chr22, position: 21411489}]"


None
	Found 1 variant associations
Gathering region data for CTA-29F11.1
	Found 2 hits


...

None
	Found 21 variant associations
Gathering region data for NDUFA6
	Found 11 hits


,,,,,,,,,,,,,,,,,,,,,,
source,feature,score,strand,frame,havana_gene,tag,transcript_type,gene_name,gene_id,transcript_status,gene_status,transcript_name,level,exon_number,gene_type,havana_transcript,transcript_id,protein_id,exon_id,ccdsid,ont,interval
str,str,float64,str,int32,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,"interval<struct{seqname: str, position: int32}>"
"""HAVANA""","""UTR""",NA,"""-""",NA,"""OTTHUMG00000151287.4""","""basic""","""protein_coding""","""NDUFA6""","""ENSG00000184983.5""","""KNOWN""","""KNOWN""","""NDUFA6-001""","""2""",NA,"""protein_coding""","""OTTHUMT00000322089.4;""","""ENST00000498737.2""","""ENSP00000418842.2""",NA,"""CCDS33656.1""",NA,"[{seqname: chr22, position: 42481529}-{seqname: chr22, position: 42482189}]"
"""HAVANA""","""exon""",NA,"""-""",NA,"""OTTHUMG00000151287.4""","""basic""","""protein_coding""","""NDUFA6""","""ENSG00000184983.5""","""KNOWN""","""KNOWN""","""NDUFA6-001""","""2""","""3""","""protein_coding""","""OTTHUMT00000322089.4;""","""ENST00000498737.2""","""ENSP00000418842.2""","""ENSE00001912575.1""","""CCDS33656.1""",NA,"[{seqname: chr22, position: 42481529}-{seqname: chr22, position: 42482318}]"
"""HAVANA""","""transcript""",NA,"""-""",NA,"""OTTHUMG00000151287.4""","""basic""","""protein_coding""","""NDUFA6""","""ENSG00000184983.5""","""KNOWN""","""KNOWN""","""NDUFA6-001""","""2""",NA,"""protein_coding""","""OTTHUMT00000322089.4;""","""ENST00000498737.2""","""ENSP00000418842.2""",NA,"""CCDS33656.1""",NA,"[{seqname: chr22, position: 42481529}-{seqname: chr22, position: 42486959}]"
"""HAVANA""","""stop_codon""",NA,"""-""",0,"""OTTHUMG00000151287.4""","""basic""","""protein_coding""","""NDUFA6""","""ENSG00000184983.5""","""KNOWN""","""KNOWN""","""NDUFA6-001""","""2""","""3""","""protein_coding""","""OTTHUMT00000322089.4;""","""ENST00000498737.2""","""ENSP00000418842.2""","""ENSE00001912575.1""","""CCDS33656.1""",NA,"[{seqname: chr22, position: 42482187}-{seqname: chr22, position: 42482189}]"
"""HAVANA""","""CDS""",NA,"""-""",0,"""OTTHUMG00000151287.4""","""basic""","""protein_coding""","""NDUFA6""","""ENSG00000184983.5""","""KNOWN""","""KNOWN""","""NDUFA6-001""","""2""","""3""","""protein_coding""","""OTTHUMT00000322089.4;""","""ENST00000498737.2""","""ENSP00000418842.2""","""ENSE00001912575.1""","""CCDS33656.1""",NA,"[{seqname: chr22, position: 42482190}-{seqname: chr22, position: 42482318}]"
"""HAVANA""","""exon""",NA,"""-""",NA,"""OTTHUMG00000151287.4""","""basic""","""protein_coding""","""NDUFA6""","""ENSG00000184983.5""","""KNOWN""","""KNOWN""","""NDUFA6-001""","""2""","""2""","""protein_coding""","""OTTHUMT00000322089.4;""","""ENST00000498737.2""","""ENSP00000418842.2""","""ENSE00002220602.1""","""CCDS33656.1""",NA,"[{seqname: chr22, position: 42483064}-{seqname: chr22, position: 42483179}]"
"""HAVANA""","""CDS""",NA,"""-""",2,"""OTTHUMG00000151287.4""","""basic""","""protein_coding""","""NDUFA6""","""ENSG00000184983.5""","""KNOWN""","""KNOWN""","""NDUFA6-001""","""2""","""2""","""protein_coding""","""OTTHUMT00000322089.4;""","""ENST00000498737.2""","""ENSP00000418842.2""","""ENSE00002220602.1""","""CCDS33656.1""",NA,"[{seqname: chr22, position: 42483064}-{seqname: chr22, position: 42483179}]"
"""HAVANA""","""CDS""",NA,"""-""",0,"""OTTHUMG00000151287.4""","""basic""","""protein_coding""","""NDUFA6""","""ENSG00000184983.5""","""KNOWN""","""KNOWN""","""NDUFA6-001""","""2""","""1""","""protein_coding""","""OTTHUMT00000322089.4;""","""ENST00000498737.2""","""ENSP00000418842.2""","""ENSE00001296436.5""","""CCDS33656.1""",NA,"[{seqname: chr22, position: 42486610}-{seqname: chr22, position: 42486826}]"


None
	Found 20 variant associations
Gathering region data for SELM
	Found 16 hits


,,,,,,,,,,,,,,,,,,,,,,
source,feature,score,strand,frame,havana_gene,tag,transcript_type,gene_name,gene_id,transcript_status,gene_status,transcript_name,level,exon_number,gene_type,havana_transcript,transcript_id,protein_id,exon_id,ccdsid,ont,interval
str,str,float64,str,int32,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,"interval<struct{seqname: str, position: int32}>"
"""HAVANA""","""UTR""",NA,"""-""",NA,"""OTTHUMG00000151207.1""","""basic""","""protein_coding""","""SELM""","""ENSG00000198832.6""","""KNOWN""","""KNOWN""","""SELM-001""","""2""",NA,"""protein_coding""","""OTTHUMT00000321788.1;""","""ENST00000400299.2""","""ENSP00000383155.2""",NA,"""CCDS43003.1""",NA,"[{seqname: chr22, position: 31500763}-{seqname: chr22, position: 31500958}]"
"""HAVANA""","""exon""",NA,"""-""",NA,"""OTTHUMG00000151207.1""","""basic""","""protein_coding""","""SELM""","""ENSG00000198832.6""","""KNOWN""","""KNOWN""","""SELM-001""","""2""","""5""","""protein_coding""","""OTTHUMT00000321788.1;""","""ENST00000400299.2""","""ENSP00000383155.2""","""ENSE00003467983.1""","""CCDS43003.1""",NA,"[{seqname: chr22, position: 31500763}-{seqname: chr22, position: 31501114}]"
"""HAVANA""","""transcript""",NA,"""-""",NA,"""OTTHUMG00000151207.1""","""basic""","""protein_coding""","""SELM""","""ENSG00000198832.6""","""KNOWN""","""KNOWN""","""SELM-001""","""2""",NA,"""protein_coding""","""OTTHUMT00000321788.1;""","""ENST00000400299.2""","""ENSP00000383155.2""",NA,"""CCDS43003.1""",NA,"[{seqname: chr22, position: 31500763}-{seqname: chr22, position: 31503554}]"
"""HAVANA""","""stop_codon""",NA,"""-""",0,"""OTTHUMG00000151207.1""","""basic""","""protein_coding""","""SELM""","""ENSG00000198832.6""","""KNOWN""","""KNOWN""","""SELM-001""","""2""","""5""","""protein_coding""","""OTTHUMT00000321788.1;""","""ENST00000400299.2""","""ENSP00000383155.2""","""ENSE00003467983.1""","""CCDS43003.1""",NA,"[{seqname: chr22, position: 31500956}-{seqname: chr22, position: 31500958}]"
"""HAVANA""","""CDS""",NA,"""-""",0,"""OTTHUMG00000151207.1""","""basic""","""protein_coding""","""SELM""","""ENSG00000198832.6""","""KNOWN""","""KNOWN""","""SELM-001""","""2""","""5""","""protein_coding""","""OTTHUMT00000321788.1;""","""ENST00000400299.2""","""ENSP00000383155.2""","""ENSE00003467983.1""","""CCDS43003.1""",NA,"[{seqname: chr22, position: 31500959}-{seqname: chr22, position: 31501114}]"
"""HAVANA""","""exon""",NA,"""-""",NA,"""OTTHUMG00000151207.1""","""basic""","""protein_coding""","""SELM""","""ENSG00000198832.6""","""KNOWN""","""KNOWN""","""SELM-001""","""2""","""4""","""protein_coding""","""OTTHUMT00000321788.1;""","""ENST00000400299.2""","""ENSP00000383155.2""","""ENSE00003494089.1""","""CCDS43003.1""",NA,"[{seqname: chr22, position: 31501194}-{seqname: chr22, position: 31501272}]"
"""HAVANA""","""CDS""",NA,"""-""",1,"""OTTHUMG00000151207.1""","""basic""","""protein_coding""","""SELM""","""ENSG00000198832.6""","""KNOWN""","""KNOWN""","""SELM-001""","""2""","""4""","""protein_coding""","""OTTHUMT00000321788.1;""","""ENST00000400299.2""","""ENSP00000383155.2""","""ENSE00003494089.1""","""CCDS43003.1""",NA,"[{seqname: chr22, position: 31501194}-{seqname: chr22, position: 31501272}]"
"""HAVANA""","""exon""",NA,"""-""",NA,"""OTTHUMG00000151207.1""","""basic""","""protein_coding""","""SELM""","""ENSG00000198832.6""","""KNOWN""","""KNOWN""","""SELM-001""","""2""","""3""","""protein_coding""","""OTTHUMT00000321788.1;""","""ENST00000400299.2""","""ENSP00000383155.2""","""ENSE00003485451.1""","""CCDS43003.1""",NA,"[{seqname: chr22, position: 31501644}-{seqname: chr22, position: 31501678}]"


None
	Found 1 variant associations


# Site metadata file
This is a metadata file containing configuration options for the TOB browser. These are used by the framework to automate table rendering and file serving on the client and server side.

Framework *requires* the keys `reference_genome`, `gene_result_analysis_groups` and `gene_group_result_field_names`. Other fields are TOB specific.

In [33]:
human_readable_labels = {
    "BimmNaive": "Bimm Naive",
    "Bmem": "B Memory",
    "CD4all": "CD4 All",
    "CD8all": "CD8 All",
    "CD8eff": "CD8 Eff",
    "CD8unknown": "CD8 Unknown",
    "DC": "DC",
    "MonoC": "Mono C",
    "MonoNC": "Mono CN",
    "NKact": "NK Act",
    "NKmat": "NK Mat",
    "Plasma": "Plasma"
}

colors = [
    '#332288',
    '#6699cc',
    '#88ccee',
    '#44aa99',
    '#117733', 
    '#999933',
    '#ddcc77',
    '#661100',
    '#cc6677',
    '#aa4466',
    '#882255',
    '#aa4499',
]

metadata = {
    "datasets": {
        "tob": {
            "reference_genome": "GRCh37",
            "gene_result_analysis_groups": ["All"],
            "gene_group_result_field_names": cell_labels,
            "gene_results_table_headings": {
                l: human_readable_labels[l] for l in cell_labels
            },
            "gene_symbols": sorted(gene_search_terms.keys()),
            "cell_labels": sorted(cell_labels),
            "cell_colors": {k: v for (k, v) in zip(cell_labels, colors)}
        }
    }
}

with open(OUTPUT_DIR / 'metadata.json', 'wt') as file:
    json.dump(metadata, file)

# Generate gene results file
Generates the gene results `JSON` file that the exome browser framework uses to render a gene results table when requesting the `/results` URL.

In [7]:
gene_results = {"results": []}
for gene in gene_search_terms.keys():
    cell_label_results = []
    for cell_label in cell_labels:
        df = pd.read_csv(DATA_DIR / f"{cell_label}_eQTLs.tsv", header=0, delimiter='\t')
        cell_label_results.append(df[df['GENE'] == gene].shape[0])
        
    metadata = gene_metadata[gene][0]
    record = [
        metadata.get('ensembl_gene_id'),
        gene,
        metadata.get('name'),
        int(metadata.get('location').split('q')[0]),
        1,
        [cell_label_results]
    ]
    
    gene_results['results'].append(record)

with open(OUTPUT_DIR / 'results'/ 'tob.json', 'wt') as file:
    json.dump(gene_results, file)

# Process expression files
Each file ending in `*_residual expression.tsv` contains expression data for a given gene listed in the columns. Each file lists different genes so we need to concatenate each dataframe initialising missing columns to `0`. This file will be served for UMAP construction. Saved in `

In [9]:
dfs = {
    label: pd.read_csv(DATA_DIR / f"{label}_residual_expressions.tsv", header=0, delimiter='\t')
    for label in cell_labels
}

genes = list(set(g for df in dfs.values() for g in df.columns ))
for gene in genes:
    for cell_label, df in dfs.items():
        df['cell_label'] = cell_label
        if gene not in df.columns:
            df[gene] = 0
            
expression = pd.concat(dfs.values(), axis=0, ignore_index=True)
expression = expression[["cell_label"] + sorted(genes)]
expression.to_csv(OUTPUT_DIR / 'results' / 'cell_label_expression.csv', index=False, sep=",")